In [129]:
import pandas as pd
import requests
import time
import re
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.DataFrame(columns= ['Name', 'Year','Wins','Champion','MVP','Scoring Leader', 
                              'Rebound Leader', 'Assist Leader', 'WS Leader','DPOY',
                              'MIP', '6MOY', 'Coach of Year'#'All-Stars', 'All-NBA',
                              #'All-Defensive', FG%', '3P%', 
                              #'2P%', 'FT%', 'PPG', 'TOV', 'AST', 'STL', 'BLK' 
                              #'SEED', 'GOAT Factor', 'Coach', 'Overall Team Score' 
                             ])
data

,Name,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,MIP,6MOY,Coach of Year


In [206]:
def check_roster(team_soup, season_awards):
    awards = [0,0,0,0,0,0,0,0,0]
    players = team_soup.find_all('td', {'class':'left', 'data-stat':'player'})
    roster = [player.text for player in players]
    coach = roster.append(team_soup.find(text=re.compile('Coach:')).parent.parent.find('a').text)

 
    for i in range(9):
        if season_awards[i] in roster:
            awards[i] = 1
    
    return awards

In [207]:
def get_team_row(season_awards, name, team_url, year, wins):
    row = []
    row.append(name.replace('*',''))
    row.append(year)
    row.append(wins)
    
    roster = []
    
    url = 'http://www.basketball-reference.com' + team_url
    req = requests.get(url)
    time.sleep(0.1)
    team_soup = BeautifulSoup(req.text, 'html.parser')
    try:
        row.append(team_soup.find('br').text[:4] == 'Won ')
    except:
        row.append(False)
    row += check_roster(team_soup, season_awards)
    
    return row

In [268]:
leaders = ['Most Valuable Player', 'PPG Leader', 'RPG Leader',
           'APG Leader', 'WS Leader']

awards = ['Defensive Player of the Year', 'Most Improved Player', 
          'Sixth Man of the Year', 'Coach of the Year']
winners = []

def get_award_winners(season, year):
    for leader in leaders:
        winners.append(season.body.find(text=leader).parent.parent.find('a').text)

    #Using a different site that's more convenient to scrape other awards.
    year_string = '%s' % str(year)
    next_year = '%s' % str(year+1)
    year = '%s-%s' % (year_string, next_year[2:4])
    for award in awards:
        url = 'http://www.nba.com/history/%s-award/' %(award.lower().replace(' ', '-'))
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'html.parser')

        table = soup.find('table')    
        try:
            winners.append(table.find(text=re.compile(year)).parent.parent.find_all('td')[1].text.strip())
        except:
            winners.append('')
            
    #Get the all stars
    all_stars =[]
    allst = season.find_all(id_="all_star_game_rosters_1")
    print()
   
    
    
    
    return winners 

In [269]:
row_num = 0
for i in range(1965, 1966):
    url = ('http://www.basketball-reference.com/leagues/NBA_%s.html' % str(i))
    time.sleep(0.1)
    req = requests.get(url)
    season = BeautifulSoup(req.text, 'html.parser')
    teams = season.find_all('th', {'class': 'left', 'data-stat':'team_name'})
   
    season_awards = get_award_winners(season, i-1)
    for team in teams:
        if(team.text == 'Team' or 'Conference' in team.text):
            continue
        wins = int(team.parent.find('td', {'data-stat':'wins'}).text)
     
        team_name = repr(team.text).replace('\\x','*').split('*')[0].lstrip("'")
        for a in team.find_all('a',href=True):
            new_row = get_team_row(season_awards,team_name,a['href'], i, wins)
  
            data.loc[row_num] = new_row
            row_num += 1


None


In [210]:
data

,Name,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,MIP,6MOY,Coach of Year
0,Boston Celtics,1965.0,62.0,True,1,0,1,0,0,0,0,0,1
1,Cincinnati Royals,1965.0,48.0,False,0,0,0,1,1,0,0,0,0
2,Philadelphia 76ers,1965.0,40.0,False,0,1,0,0,0,0,0,0,0
3,New York Knicks',1965.0,31.0,False,0,0,0,0,0,0,0,0,0
4,Los Angeles Lakers,1965.0,49.0,False,0,0,0,0,0,0,0,0,0
5,St. Louis Hawks,1965.0,45.0,False,0,0,0,0,0,0,0,0,0
6,Baltimore Bullets,1965.0,37.0,False,0,0,0,0,0,0,0,0,0
7,Detroit Pistons',1965.0,31.0,False,0,0,0,0,0,0,0,0,0
8,San Francisco Warriors',1965.0,17.0,False,0,1,0,0,0,0,0,0,0
9,Atlanta Hawks,1985.0,34.0,False,False,False,False,False,False,False,False,False,False
